# I. Random Graph Model Testing

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

import logging
logging.captureWarnings(True) # Logging warnings in order to suppress them

In [7]:
# For this task use the full weight matrix V.
adj_matrix = np.genfromtxt("data/sgb128Nodes41to80_weight.txt", delimiter=' ', skip_header=1)
V = nx.Graph(adj_matrix)